In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [122]:
def refresh_authetication():
    ## Connect to spotify
    CLIENT_ID = 'f4b0975d0a7e419fb4ef187ddd061914'
    CLIENT_SECRET = '7565d5fa608c4656a304af75c0993ba3'
    REDIRECT_URI = 'http://localhost/'
    #REDIRECT_URI = 'localhost:8080'
    USERNAME = '31dmwz4yaka4jssvnhebn4dsnn2a'
    SCOPES = [
        'user-modify-playback-state',
        'user-read-playback-state',
        'user-read-currently-playing',
        'user-follow-modify',
        'user-follow-read',
        'user-read-recently-played',
        'user-read-playback-position',
        'user-top-read',
        'playlist-read-collaborative',
        'playlist-modify-public',
        'playlist-read-private',
        'playlist-modify-private',
        'app-remote-control',
        'streaming',
        'user-library-modify',
        'user-library-read'
    ]
    token = spotipy.util.prompt_for_user_token(USERNAME,SCOPES,client_id=CLIENT_ID,client_secret=CLIENT_SECRET,redirect_uri=REDIRECT_URI)
    return spotipy.Spotify(auth=token)

token = 'AQCIOQNMN2dZHPaP3uBckk9PeUZEHGdOtlHxyOC24MWz8MmMXHyQtWNIU3lohaEdy-vWDPCbp2FJQrVTn2ZAFsPzuxqNcc1iEOQGst3vC5GdGidSipGSbij7V6VmNrdD3Obt8RJrkcbXXIo4pB6UZWH01ucxAHKkFUa3DYpGZEVyobVxFnBHo8CcTePMLqwH7sbTmGZtqM4QffTdqHYhnCMMPoa4blBbY7mRXx7J0Bfh7UL_O1QCz2BcgEpuRSNMJsVRRU2F8g9LsLsVspvDTVKu4IP8WeX8yHD22DhVYtKKjkxaOpvYMFHfVkjf-V2xl-7atwHPQStBVKhPO-WuqTNFeUXw6JPt2zhBXFh_od66zFvYjbUQz1etL7QriQ0rDL3yYpVw0sILMPlVBWz8RpUGy0u531LxIqxfsJWjZvKL5HV1x9O4oNA65IlUoMTkyEZ4EDvFdLTs7UIK97p3D6QiTPjLvxzVDXwdTydGQFFexU2S4Nw44z0uXsqYHLucGjGCrPkpFULWzF4EZ1FYjp9befLTWWA_DrgfpeAqFwkF7p1FtyGL-IV6lqO3hRimapjUttp9Sf8AAGs5DEcAP4hu2U5wsYxraDrmq7tF7hx7px15UA9OSORO_9Ya_3VIUA'
sp = spotipy.Spotify(auth=token)
##refresh_authetication()

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def client_auth_flow():
    # CLIENT_ID = '6a253a1731a048f7a1c91c1145210117'
    # CLIENT_SECRET = '92eff559527a413696007220dd9ff832'
    CLIENT_ID = '59d4c64173a6440a87d0a173e1a13a73'
    CLIENT_SECRET = '7e685da85a094516b5fb271568ec2abf'
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [50]:
# read rym data
rym_df = pd.read_csv('../data/rym-1950-2021.csv', sep='~')
rym_df.tail(10)

,Unnamed: 0.1,Unnamed: 0,Rank,Artist,Album,Date,Genres,RYM Rating,Ratings,Reviews
2470,950,30,31,Nas,Magic,24 December 2021,"East Coast Hip Hop, Boom Bap",3.67,3041.0,39.0
2471,951,31,32,Effluence,Psychocephalic Spawning,13 December 2021,"Avant-Garde Metal, Brutal Death Metal, Technic...",3.63,480.0,11.0
2472,952,32,33,"岡部啓一 [Keiichi Okabe], 帆足圭吾 [Keigo Hoashi], 石濱翔...",NieR Replicant ver.1.22474487139...,21 April 2021,"Video Game Music, Cinematic Classical, Neoclas...",3.61,407.0,5.0
2473,953,33,34,Wild Up,Julius Eastman Vol. 1: Femenine,18 June 2021,"Minimalism, Chamber Music",3.67,1290.0,9.0
2474,954,34,35,Ad Nauseam,Imperative Imperceptible Impulse,12 February 2021,"Technical Death Metal, Avant-Garde Metal",3.63,2620.0,30.0
2475,955,35,36,DJ Sabrina the Teenage DJ,The Makin' Magick II Album,26 November 2021,Outsider House,3.65,1099.0,4.0
2476,956,36,37,STOMACH BOOK,STOMACH BOOK,8 October 2021,"Experimental Rock, Noise Pop, Emo",3.66,606.0,2.0
2477,957,37,38,Skee Mask,Pool,7 May 2021,"IDM, Breakbeat, Techno",3.64,2604.0,13.0
2478,958,38,39,McKinley Dixon,For My Mama and Anyone Who Look Like Her,7 May 2021,"Jazz Rap, Southern Hip Hop, Conscious Hip Hop",3.64,1773.0,21.0
2479,959,39,40,Jaubi,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0


In [40]:
# processing to check similar album titles

from collections import defaultdict

SIM_THRES = 0.2
NOT_FOUND = 'NOT_FOUND'

def get_word_freq(l):
    d = defaultdict(int)
    for w in l:
        d[w] += 1
    return d

def freq_match(d1,d2):
    n = len(d1)
    sim = 0
    for k,v in d1.items():
        if k in d2 and d2[k] == v:
            sim += 1
    return sim/n


def are_same_albums(s1,s2):
    s1 = s1.lower()
    s2 = s2.lower()
    freq1 = get_word_freq(s1.split(' '))
    freq2 = get_word_freq(s2.split(' '))
    sim1 = freq_match(freq1, freq2)
    sim2 = freq_match(freq2, freq1)
    if max(sim1,sim2) >= SIM_THRES:
        return True
    return False

In [34]:
# retrieve spotify album id from a given name
def get_spotify_album(name): 
    search_res = sp.search(q=name, type='album')
    if len(search_res['albums']['items']) > 0 and are_same_albums(search_res['albums']['items'][0]['name'], name):
        return search_res['albums']['items'][0]['uri']
    return NOT_FOUND

In [35]:
# retrieve tracks of a valid album id
def get_album_tracks(album_id):
    tracks = []
    for track in sp.album_tracks(album_id=album_id)['items']:
        tracks.append(track['uri'])
    return tracks

In [36]:
# retrieve audio features
def get_audio_features(tracks):
    return sp.audio_features(tracks)

# retrieve audio analysis
def get_audio_analysis(track_id):
    return sp.audio_analysis(track_id)

In [37]:
# columns from different data sources
audio_features_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
song_meta_cols = ['name', 'duration_ms', 'popularity', 'type']
rym_columns = ['Album', 'Date', 'Genres', 'RYM Rating', 'Ratings', 'Reviews']

In [51]:
rym_df["d"] = rym_df["Date"].apply(convert_dt)

In [88]:
from datetime import datetime

#rym_df = rym_df[rym_df["d"] > datetime.strptime('1994-11-08',"%Y-%m-%d")]
#rym_df = rym_df[rym_df["d"] > datetime.strptime('2000-02-08',"%Y-%m-%d")]
#rym_df = rym_df[rym_df.index > 1732]
rym_df.head()

,Unnamed: 0.1,Unnamed: 0,Rank,Artist,Album,Date,Genres,RYM Rating,Ratings,Reviews,d
1733,213,13,14,Opeth,Damnation,22 April 2003,Progressive Rock,3.84,10639.0,233.0,2003-04-22
1734,214,14,15,目黒将司 [Shoji Meguro] / 田崎寿子 [Toshiko Tasaki] / ...,真・女神転生III Nocturne (Shin Megami Tensei III: No...,5 March 2003,"Video Game Music, Sequencer & Tracker",3.92,400.0,1.0,2003-03-05
1735,215,15,16,大谷幸 [Ko Otani],Haibane-Renmei: Hanenone,19 August 2003,"Television Music, Chamber Music",3.92,377.0,4.0,2003-08-19
1736,216,16,17,Sleep,Dopesmoker,22 April 2003,"Stoner Metal, Doom Metal",3.82,9575.0,144.0,2003-04-22
1737,217,17,18,"永田権太 [Kenta Nagata], 若井淑 [Hajime Wakai], 峰岸透 [...",ゼルダの伝説 ～風のタクト～ (The Legend of Zelda: The Wind ...,19 March 2003,"Video Game Music, Sequencer & Tracker",3.89,601.0,5.0,2003-03-19


In [90]:
# get audio features
track_features = []

def add_keys(dfrom, dto, keys):
    for key in keys:
        if key in dfrom:
            dto[key.lower()] = dfrom[key]
        else:
            dto[key.lower()] = None

switch = False
client_auth_flow()
for index, row in rym_df.iterrows():
    if(index%500 == 0):
        client_auth_flow()
    if index > 1732:
        album_id = get_spotify_album(row['Album'])
        if album_id != NOT_FOUND:
            tracks = get_album_tracks(album_id=album_id)
            curr_features = sp.audio_features(tracks)
            for d in curr_features:
                if d and set(audio_features_cols) == set(d.keys()):
                    # add rym columns
                    add_keys(row, d, rym_columns)
                    # add song metadata
                    song_meta = sp.track(d['id'])
                    add_keys(song_meta, d, song_meta_cols)
                    track_features.append(d)
                    did,aid = d['id'], row['Album']
                    print(f'Songs added for {did}, {aid}')

Songs added for 4uJSCrI7r0usNJ3aaHAuC6, Damnation
Songs added for 3WtsQAngIkmcNeqVeK3fXQ, Damnation
Songs added for 61Hd6XNqSpreYVEiMkaHvq, Damnation
Songs added for 0hF30hdCIcjXmd8gamwOYU, Damnation
Songs added for 05lem7bHWxyo98HOwPmils, Damnation
Songs added for 6lKG104xIQ7DIuSYOnCqfZ, Damnation
Songs added for 2wXSx9UxsNIYidGi5r0SZ4, Damnation
Songs added for 5c1Fu2ETbvOhMV9NL3UkS5, Damnation
Songs added for 4KdQgYM9LkpACLauqsN6tD, 真・女神転生III Nocturne (Shin Megami Tensei III: Nocturne)
Songs added for 5bu9A6uphPWg39RC3ZKeku, Variations Goldberg BWV 988
Songs added for 14V6VqJXHZ08pM6Fu3vUsO, Variations Goldberg BWV 988
Songs added for 2mXVCqoNE6otHBcmRz4IHv, Variations Goldberg BWV 988
Songs added for 5SJFZnECoKeTLy7PPVLLbF, Variations Goldberg BWV 988
Songs added for 14lyZKpPu1ECQpfXhgrip6, Variations Goldberg BWV 988
Songs added for 325xUomC1NKDerrKIn0zfD, Variations Goldberg BWV 988
Songs added for 0L5cUf2OgB2GeVzxwGJv3q, Variations Goldberg BWV 988
Songs added for 4w3aG0qWqCQKhv

In [92]:
track_features

[{'danceability': 0.328,
  'energy': 0.553,
  'key': 6,
  'loudness': -6.252,
  'mode': 1,
  'speechiness': 0.0267,
  'acousticness': 0.156,
  'instrumentalness': 0.412,
  'liveness': 0.0985,
  'valence': 0.322,
  'tempo': 159.959,
  'type': 'track',
  'id': '4uJSCrI7r0usNJ3aaHAuC6',
  'uri': 'spotify:track:4uJSCrI7r0usNJ3aaHAuC6',
  'track_href': 'https://api.spotify.com/v1/tracks/4uJSCrI7r0usNJ3aaHAuC6',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4uJSCrI7r0usNJ3aaHAuC6',
  'duration_ms': 464280,
  'time_signature': 4,
  'album': 'Damnation',
  'date': '22 April 2003',
  'genres': 'Progressive Rock',
  'rym rating': 3.84,
  'ratings': 10639.0,
  'reviews': 233.0,
  'name': 'Windowpane',
  'popularity': 53},
 {'danceability': 0.471,
  'energy': 0.604,
  'key': 4,
  'loudness': -6.014,
  'mode': 0,
  'speechiness': 0.0286,
  'acousticness': 0.00161,
  'instrumentalness': 0.154,
  'liveness': 0.166,
  'valence': 0.187,
  'tempo': 137.388,
  'type': 'track',
  'id': '3Wt

In [91]:
pd.DataFrame(track_features).tail()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,duration_ms,time_signature,album,date,genres,rym rating,ratings,reviews,name,popularity
8182,0.271,0.801,0,-7.291,0,0.1050,0.00096,0.7010,0.0607,0.0947,...,373613,4,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0,Raga Gujri Todi,22
8183,0.411,0.519,6,-11.562,0,0.0473,0.19000,0.1570,0.0845,0.2110,...,447160,4,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0,Straight Path,23
8184,0.552,0.802,10,-8.149,0,0.0293,0.00746,0.7550,0.3180,0.5870,...,460965,4,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0,Mosty,31
8185,0.541,0.667,2,-8.078,1,0.0318,0.00416,0.0922,0.3360,0.2740,...,351461,4,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0,Zari,27
8186,0.150,0.695,8,-6.390,0,0.0531,0.00115,0.3510,0.1160,0.2830,...,474372,3,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0,Nafs at Peace,18


In [93]:
df2 = pd.DataFrame(track_features)
df2.to_csv('../data/merge-data-3.csv', sep='~')

In [187]:
columns = []
tabular_features = defaultdict(list)
for track in track_features:
    if len(track) > 0 and track[0]:
        for k,v in track[0].items():
            tabular_features[k].append(v)
song_feature_df = pd.DataFrame(tabular_features)
song_feature_df.tail()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
1779,0.398,0.655,0,-10.301,1,0.0500,0.0261,0.000082,0.5670,0.1460,90.000,audio_features,7u3s0S0Xn4NNezPTtnKrkw,spotify:track:7u3s0S0Xn4NNezPTtnKrkw,https://api.spotify.com/v1/tracks/7u3s0S0Xn4NN...,https://api.spotify.com/v1/audio-analysis/7u3s...,438497,4
1780,0.376,0.868,1,-5.766,1,0.0878,0.1080,0.000112,0.1690,0.4090,99.777,audio_features,4Nv0jpNVsCMgDnhOlq8cUm,spotify:track:4Nv0jpNVsCMgDnhOlq8cUm,https://api.spotify.com/v1/tracks/4Nv0jpNVsCMg...,https://api.spotify.com/v1/audio-analysis/4Nv0...,194400,4
1781,0.664,0.477,0,-10.530,1,0.0551,0.3940,0.000041,0.1360,0.2730,100.046,audio_features,1ZP4Qxb7USL3EezbxfpOVF,spotify:track:1ZP4Qxb7USL3EezbxfpOVF,https://api.spotify.com/v1/tracks/1ZP4Qxb7USL3...,https://api.spotify.com/v1/audio-analysis/1ZP4...,223000,4
1782,0.414,0.800,6,-6.154,0,0.1230,0.1990,0.000000,0.3160,0.4270,128.647,audio_features,26NekpDz9TlmOzB2KtX3Kx,spotify:track:26NekpDz9TlmOzB2KtX3Kx,https://api.spotify.com/v1/tracks/26NekpDz9Tlm...,https://api.spotify.com/v1/audio-analysis/26Ne...,280955,4
1783,0.165,0.197,6,-16.400,1,0.0346,0.9420,0.498000,0.0895,0.0749,100.856,audio_features,3aD10lO7Runy5cpN9YaoNz,spotify:track:3aD10lO7Runy5cpN9YaoNz,https://api.spotify.com/v1/tracks/3aD10lO7Runy...,https://api.spotify.com/v1/audio-analysis/3aD1...,138670,4


In [198]:
audio_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
rym_columns = ['Album', 'Date', 'Genres', 'RYM Rating', 'Ratings', 'Reviews']

In [204]:
print(f'Songs added for {columns}')

Songs added for ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']


In [208]:
rym_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Rank', 'Artist', 'Album', 'Date',
       'Genres', 'RYM Rating', 'Ratings', 'Reviews'],
      dtype='object')

In [101]:
## get spotify genres
df = pd.read_csv('../data/merge-data-3.csv', sep='~')

In [102]:
df.columns

Index(['Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'album', 'date', 'genres',
       'rym rating', 'ratings', 'reviews', 'name', 'popularity'],
      dtype='object')

In [138]:
album_id1 = get_spotify_album('The Dark Side of the Moon')
tracks1 = get_album_tracks(album_id=album_id1)
tracks_feature1 = get_audio_features(tracks1)
client_auth_flow()
temp = sp.track(tracks1[0])

KeyboardInterrupt: 

In [5]:
sp = client_auth_flow()
sp.track('4uJSCrI7r0usNJ3aaHAuC6')

KeyboardInterrupt: 

In [105]:
df['rym_genres'] = df['genres'][:]
df = df.drop(['genres'], axis=1)
df.columns

Index(['Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'album', 'date', 'rym rating',
       'ratings', 'reviews', 'name', 'popularity', 'rym_genres'],
      dtype='object')

In [106]:
# convert str date to datetime
import dateutil.parser

def convert_dt(s):
    return dateutil.parser.parse(s)
df["date"] = df["date"].apply(convert_dt)

In [57]:
df1["date"] = df1["date"].apply(convert_dt)
# rt = df.tail()
# type(rt["date"].loc[14358])

In [107]:
def get_genres(track_id):
    try:
        return sp.artist(sp.track(track_id)['artists'][0]['id'])['genres']
    except Exception:
        return []
genre_dat = []
for idx, track_id in enumerate(df["id"]):
    if(idx%500 == 0):
        sp = client_auth_flow()
    if(idx > 434):
        curr_genre = get_genres(track_id)
        print(idx, curr_genre)
        genre_dat.append(curr_genre)
#df['sp_genres'] = df['id'].apply(get_genres)

0 ['alternative metal', 'metal', 'progressive death metal', 'progressive metal', 'rock', 'swedish metal', 'swedish progressive metal']
1 ['alternative metal', 'metal', 'progressive death metal', 'progressive metal', 'rock', 'swedish metal', 'swedish progressive metal']
2 ['alternative metal', 'metal', 'progressive death metal', 'progressive metal', 'rock', 'swedish metal', 'swedish progressive metal']
3 ['alternative metal', 'metal', 'progressive death metal', 'progressive metal', 'rock', 'swedish metal', 'swedish progressive metal']
4 ['alternative metal', 'metal', 'progressive death metal', 'progressive metal', 'rock', 'swedish metal', 'swedish progressive metal']
5 ['alternative metal', 'metal', 'progressive death metal', 'progressive metal', 'rock', 'swedish metal', 'swedish progressive metal']
6 ['alternative metal', 'metal', 'progressive death metal', 'progressive metal', 'rock', 'swedish metal', 'swedish progressive metal']
7 ['alternative metal', 'metal', 'progressive death met

KeyboardInterrupt: 

In [108]:
len(genre_dat)

434

In [109]:
pd.Series(genre_dat).to_csv('../data/genre-dat-merge-data-3-434.csv')

In [64]:
pd.concat([df, df1], ignore_index=True)

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,time_signature,album,date,rym rating,ratings,reviews,name,popularity,rym_genres,genres
0,0.0,0.192,0.0409,0,-29.467,1,0.0391,0.994,0.980000,0.0956,...,4,Giant Steps,1960-01-27,4.11,14738.0,198.0,Giant Steps Tomorrow,64,Hard Bop,NaN
1,1.0,0.364,0.4700,5,-13.051,0,0.0730,0.687,0.002540,0.0849,...,4,Blues & Roots,1960-03-26,4.04,6610.0,116.0,Wednesday Night Prayer Meeting,31,Post-Bop,NaN
2,2.0,0.518,0.0804,8,-20.904,1,0.0434,0.752,0.072100,0.0935,...,4,Blues & Roots,1960-03-26,4.04,6610.0,116.0,Cryin' Blues,27,Post-Bop,NaN
3,3.0,0.488,0.4760,5,-10.316,0,0.0497,0.438,0.000279,0.1100,...,4,Blues & Roots,1960-03-26,4.04,6610.0,116.0,Moanin',47,Post-Bop,NaN
4,4.0,0.560,0.2760,8,-17.947,1,0.0628,0.625,0.005390,0.1060,...,4,Blues & Roots,1960-03-26,4.04,6610.0,116.0,Tensions,26,Post-Bop,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17214,NaN,0.506,0.7730,1,-6.376,0,0.2840,0.164,0.000000,0.2960,...,4,Supreme Clientele,2000-02-08,3.89,7137.0,116.0,Stay True (feat. 60 Second Assassin),28,NaN,"East Coast Hip Hop, Hardcore Hip Hop, Abstract..."
17215,NaN,0.699,0.8620,9,-6.117,0,0.2790,0.108,0.000000,0.3770,...,4,Supreme Clientele,2000-02-08,3.89,7137.0,116.0,We Made It (feat. Superb),29,NaN,"East Coast Hip Hop, Hardcore Hip Hop, Abstract..."
17216,NaN,0.624,0.6700,5,-5.406,0,0.3420,0.311,0.000000,0.0616,...,4,Supreme Clientele,2000-02-08,3.89,7137.0,116.0,Stroke of Death,29,NaN,"East Coast Hip Hop, Hardcore Hip Hop, Abstract..."
17217,NaN,0.576,0.3760,10,-11.828,0,0.0564,0.669,0.000019,0.6710,...,4,Supreme Clientele,2000-02-08,3.89,7137.0,116.0,Iron's Theme (Intermission),25,NaN,"East Coast Hip Hop, Hardcore Hip Hop, Abstract..."


In [54]:
rt[rt["date"] >= ]

14358    8 November 1994
14359    8 November 1994
14360    8 November 1994
14361    8 November 1994
14362    8 November 1994
Name: date, dtype: object

pandas._libs.tslibs.timestamps.Timestamp

In [61]:
type(rt["date"][14358])

str

In [132]:
rt["id"][14358]